In [1]:
from torch_geometric.utils import to_dgl, stochastic_blockmodel_graph
from torch_geometric.data import Data
import torch
import numpy as np
import sys
sys.path.append("../")


x_train = torch.rand(590,9)

d = 4

n_P1 = 100 # NUMERO DE SENADORES P1
n_P2 = 80 # NUMERO DE SENADORES P2
n_L1 = 200 # NUMERO DE LEYES P1
n_L2 = 150 # NUMERO DE LEYES P2
n_L3 = 60 # NUMERO DE LEYES NEUTRAS

P1_L1 = 0.9 ## Votos de senadores del partido 1 a leyes grupo 1
P1_L2 = 0.01 ## Votos de senadores del partido 1 a leyes grupo 2
P1_L3 = 0.2 ## Votos de senadores del partido 1 a leyes grupo 3
P2_L1 = 0.1 ## Votos de senadores del partido 2 a leyes grupo 1
P2_L2 = 0.8 ## Votos de senadores del partido 2 a leyes grupo 2
P2_L3 = 0.3 ## Votos de senadores del partido 2 a leyes grupo 3

p = [
    [0, 0, P1_L1, P1_L2, P1_L3],
    [0, 0, P2_L1, P2_L2, P2_L3],
    [P1_L1, P2_L1, 0, 0, 0], 
    [P1_L2, P2_L2, 0, 0, 0], 
    [P1_L3, P2_L3, 0, 0, 0]
    ]

n = [n_P1, n_P2, n_L1, n_L2, n_L3]

num_nodes = np.sum(n)
edge_index = stochastic_blockmodel_graph(n, p)

data = Data(x=x_train, edge_index=edge_index)
g = to_dgl(data)


In [2]:
from models.link_prediction import GraphTransformerLinkPrediction, train, test

def train_link_prediction_GraphTransformer(x_train, x_val, x_test, train_data, val_data, test_data, input_dim, epochs = 101): 
    device = "cuda"
    model = GraphTransformerLinkPrediction(input_dim, 128, 64, n_layers=3, dropout=0.5, num_heads=1).to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
    criterion = torch.nn.BCEWithLogitsLoss()
    best_val_auc = final_test_auc = 0
    for epoch in range(1, epochs):
        loss = train(x_train, train_data, model, optimizer, criterion)
        val_auc = test(x_val, val_data, model)
        test_auc = test(x_test, test_data, model)
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            final_test_auc = test_auc
    return model


In [6]:
model = GraphTransformerLinkPrediction(9, 64, 64, n_layers=10, dropout=0.5, num_heads=4).to('cuda')

model

GraphTransformerLinkPrediction(
  (layers): ModuleList(
    (0): GraphTransformerLayer(in_channels=9, out_channels=64, heads=4, residual=False)
    (1-9): 9 x GraphTransformerLayer(in_channels=64, out_channels=64, heads=4, residual=True)
  )
)

In [8]:
loss = train(x_train, data, model, optimizer, criterion)

NameError: name 'optimizer' is not defined